<h1>국내 여행 로그 데이터(서부권) (2023)</h1>
<h2>[데이터 정보]</h2>
<ul>
<li>2023 12,800명 이상의 데이터 구축</li>
<li>Metadata 관련 파일 : 시군구코드.csv, 코드A.csv, 코드B.csv (분석 참고용) </li>
<li>분석용 데이터 : 여행객.csv, 동반자정보.csv, 여행.csv, 이동내역.csv, GPS좌표.csv, 이동수단소비내역.csv, 숙박소비내역.csv, 활동내역.csv, 사전소비내역.csv, 활동소비내역.csv, 방문지정보.csv, 관광사진.csv
<br> > 참고 : 가독성을 위해 원본 데이터의 파일명(CSV명)을 일부 수정하였고, 분석에 필요하지 않다고 판단한 GPS좌표.csv, 관광사진.csv, 이동내역.csv는 제외하고 분석을 진행</li>
<li> Metadata 관련 파일 설명
<br> 1. 시군구코드.csv
 <br>- 시도코드(2자리), 시군구코드(3자리), 읍면동코드(3자리)를 포함
 <br>- 시군구코드가 필요한 분석 결과는 해당 CSV 파일을 참고
<br> 2. 코드B.csv
 <br>- 분석용 데이터에서 숫자값으로 입력된 코드들의 실제 한글값을 제공
 <br>- 해당 분석에서는 시각화 시 가독성을 위해 코드B.csv의 값들을 이용하여 숫자값을 한글값으로 변환하여 사용</li></ul>
<h2>[분석 정보]</h2>
<ul>
<li>원본 데이터 손상 방지를 위해 모든 데이터는 복사본을 만들어서 분석 진행
<br> > 예시 : 원본 데이터 프레임이 raw_traveller_master 라면, 분석을 진행하는 데이터 프레임은 traveller_master로 설정</li>
<li>각 CSV 파일에서 분석에 필요한 변수들은 사전 작업을 통해 선별함
<br>의미 및 순위가 불확실한 변수들도 제외</li>
<li>여행객.csv를 제외한 모든 CSV 파일에는 TRAVEL_ID가존재하며, 여행객.csv의 TRAVELER_ID도 TRAVEL_ID의 형식으로 변환 가능하므로, TRAVEL_ID를 기준으로 각 데이터를 통합함</li></ul>



<hr>
<h2>[패키지 및 데이터 로드]</h2>

In [1593]:
# 데이터를 분석하는데 필요한 패키지 로드하기 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
plt.rcParams.update({'font.family' : 'Malgun Gothic'}) 

# 숫자 표기 설정
pd.options.display.float_format = '{:,.2f}'.format # 지수 없이 숫자로
#pd.reset_option('display.float_format') # 지수표기법

# 출력 설정 변경 (None : 모든 행,열 출력)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
#pd.reset_option('all') # 설정 변경 초기화 

In [1594]:
# 분석용 데이터 불러오기
raw_traveller_master = pd.read_csv('data/training/여행객.csv', encoding='UTF-8')
raw_companion_info = pd.read_csv('data/training/동반자정보.csv', encoding='UTF-8')
raw_travel = pd.read_csv('data/training/여행.csv', encoding='UTF-8')
raw_mvmn_consume = pd.read_csv('data/training/이동수단소비내역.csv', encoding='UTF-8')
raw_lodge_consume = pd.read_csv('data/training/숙박소비내역.csv', encoding='UTF-8')
raw_activity = pd.read_csv('data/training/활동내역.csv', encoding='UTF-8')
raw_activity_consume = pd.read_csv('data/training/활동소비내역.csv', encoding='UTF-8')
raw_adv_consume = pd.read_csv('data/training/사전소비내역.csv', encoding='UTF-8')
raw_visit_area = pd.read_csv('data/training/방문지정보.csv', encoding='UTF-8')

# 복사본 만들기 
traveller_master = raw_traveller_master.copy()
companion_info = raw_companion_info.copy()
travel = raw_travel.copy()
mvmn_consume = raw_mvmn_consume.copy()
lodge_consume = raw_lodge_consume.copy()
activity = raw_activity.copy()
activity_consume = raw_activity_consume.copy()
adv_consume = raw_adv_consume.copy()
visit_area = raw_visit_area.copy()

<hr>
<h2>[한글값 매핑]</h2>

In [1595]:
# 코드 'ACT'
ACT_mapping = {
    1 : '취식',
    2 : '쇼핑 / 구매',
    3 : '체험 활동 / 입장 및 관람',
    4 : '단순 구경 / 산책 / 걷기',
    5 : '휴식',
    6 : '기타 활동',
    7 : '환승/경유',
    99 : '없음'
}

# 코드 'AGE'
AGE_mapping = {
    1 : '~9세이하',
    2 : '10대',
    3 : '20대',
    4 : '30대',
    5 : '40대',
    6 : '50대',
    7 : '60대',
    8 : '70세이상'
}

# 코드 'GEN'
GEN_mapping = {
    1 : '남자',
    2 : '여자'
}

# 코드 'HTY'
HTY_mapping = {
    1 : '호텔',
    2 : '모텔/여관',
    3 : '펜션',
    4 : '콘도미니엄/리조트',
    5 : '야영(캠핑,글램핑등)',
    6 : '전통 숙박시설',
    7 : '게스트하우스',
    8 : '민박',
    9 : '자연휴양림내 시설',
    10 : '가족/친지집',
    11 : '기타',
    12 : '무박 여행'
}

# 코드 'JOB'
JOB_mapping = {
    1 : '관리자',
    2 : '전문가 및 관련 종사자',
    3 : '사무 종사자',
    4 : '서비스 종사자',
    5 : '판매 종사자',
    6 : '농림어업 숙련 종사자',
    7 : '기능원 및 관련 기능 종사자',
    8 : '장치․기계 조작 및 조립 종사자',
    9 : '단순노무종사자',
    10 : '군인',
    11 : '전업주부',
    12 : '학생',
    13 : '기타'
}

# 코드 'MIS'
MIS_mapping = {
    1 : '쇼핑',
    2 : '테마파크, 놀이시설, 동/식물원 방문',
    3 : '역사 유적지 방문',
    4 : '시티투어',
    5 : '야외 스포츠, 레포츠 활동',
    6 : '지역 문화예술/공연/전시시설 관람',
    7 : '유흥/오락(나이트라이프)',
    8 : '캠핑',
    9 : '지역 축제/이벤트 참가',
    10 : '온천/스파',
    11 : '교육/체험 프로그램 참가',
    12 : '드라마 촬영지 방문',
    13 : '종교/성지 순례',
    21 : 'Well-ness 여행',
    22 : 'SNS 인생샷 여행',
    23 : '호캉스 여행',
    24 : '신규 여행지 발굴',
    25 : '반려동물 동반 여행',
    26 : '인플루언서 따라하기 여행',
    27 : '친환경 여행(플로깅 여행)',
    28 : '등반 여행'
}

# 코드 'TCR'
TCR_mapping = {
    1 : '배우자',
    2 : '자녀',
    3 : '부모',
    4 : '조부모',
    5 : '형제/자매',
    6 : '친인척',
    7 : '친구',
    8 : '연인',
    9 : '동료',
    10 : '친목 단체/모임(동호회, 종교단체 등)',
    11 : '기타'
}

# 코드 'TMT'
TMT_mapping = {
    1 : '일상적인 환경 및 역할에서의 탈출, 지루함 탈피',
    2 : '쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식',
    3 : '여행 동반자와의 친밀감 및 유대감 증진',
    4 : '진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기',
    5 : 'SNS 사진 등록 등 과시',
    6 : '운동, 건강 증진 및 충전',
    7 : '새로운 경험 추구',
    8 : '역사 탐방, 문화적 경험 등 교육적 동기',
    9 : '특별한 목적(칠순여행, 신혼여행, 수학여행, 인센티브여행)',
    10 : '기타'
}

# 코드 'TSY' (자연 VS 도시)
TSY_mapping = {
    1 : '자연 선호(강)',
    2 : '자연 선호(중)',
    3 : '자연 선호(약)',
    4 : '중립',
    5 : '도시 선호(약)',
    6 : '도시 선호(중)',
    7 : '도시 선호(강)'
}

# 코드 'TTM'
TTM_mapping = {
    1 : '1주일',
    2 : '한달',
    3 : '1년',
    4 : '기타'
}

In [1596]:
# 데이터프레임 변환 함수
def convert_codes_to_korean(df, column_name, mapping) : 
    if column_name in df.columns : 
        df[column_name] = df[column_name].astype('int64')
        df[column_name] = df[column_name].map(mapping)
    return df

<hr>
<h2>[여행객.csv]</h2>

<h3>1. 변수 추출 및 검토</h3>

In [1597]:
# 필요한 변수 리스트
columns_to_keep_traveller = [
    'TRAVELER_ID', 'GENDER', 'AGE_GRP', 'JOB_NM', 'TRAVEL_TERM', 'TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SGG_2', 'TRAVEL_LIKE_SGG_3', 
    'TRAVEL_STYL_1','TRAVEL_STYL_2','TRAVEL_STYL_3','TRAVEL_STYL_4','TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION', 'TRAVEL_STATUS_ACCOMPANY', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM'
    ]

# traveller_master에 필요한 변수만 남기고 저장
traveller_master = traveller_master[columns_to_keep_traveller]

# 변수명 변경
traveller_master = traveller_master.rename(columns={'TRAVEL_MOTIVE_1' : 'TRAVEL_MOTIVE'})

# 결과 확인
print(traveller_master.head())

  TRAVELER_ID GENDER  AGE_GRP  JOB_NM  TRAVEL_TERM  TRAVEL_LIKE_SGG_1  TRAVEL_LIKE_SGG_2  TRAVEL_LIKE_SGG_3  TRAVEL_STYL_1  TRAVEL_STYL_2  TRAVEL_STYL_3  TRAVEL_STYL_4  TRAVEL_STYL_5  TRAVEL_STYL_6  TRAVEL_STYL_7  TRAVEL_STYL_8 TRAVEL_STATUS_RESIDENCE TRAVEL_STATUS_DESTINATION TRAVEL_STATUS_ACCOMPANY  TRAVEL_MOTIVE  TRAVEL_COMPANIONS_NUM
0     g002178      여       20      12            3              50130              50110              26500              1              1              1              3              3              2              6              6                   광주광역시                        전남             2인 여행(가족 외)              1                      1
1     g004485      남       40       2            2              48840              48220              26500              1              1              3              6              4              2              7              7                   대전광역시                        전북             2인 여행(가족 외)              1    

In [1598]:
# 데이터의 행, 열 개수 출력
print('여행객.csv의 행, 열 개수 : ')
print(traveller_master.shape)

여행객.csv의 행, 열 개수 : 
(2560, 21)


In [1599]:
# 데이터의 구조와 타입 확인
print('여행객.csv의 구조와 타입 : ')
print(traveller_master.info())

여행객.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2560 entries, 0 to 2559
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   TRAVELER_ID                2560 non-null   object
 1   GENDER                     2560 non-null   object
 2   AGE_GRP                    2560 non-null   int64 
 3   JOB_NM                     2560 non-null   int64 
 4   TRAVEL_TERM                2560 non-null   int64 
 5   TRAVEL_LIKE_SGG_1          2560 non-null   int64 
 6   TRAVEL_LIKE_SGG_2          2560 non-null   int64 
 7   TRAVEL_LIKE_SGG_3          2560 non-null   int64 
 8   TRAVEL_STYL_1              2560 non-null   int64 
 9   TRAVEL_STYL_2              2560 non-null   int64 
 10  TRAVEL_STYL_3              2560 non-null   int64 
 11  TRAVEL_STYL_4              2560 non-null   int64 
 12  TRAVEL_STYL_5              2560 non-null   int64 
 13  TRAVEL_STYL_6              2560 non-null   i

In [1600]:
# 데이터의 결측치 확인
print('여행객.csv의 결측치 : ')
print(traveller_master.isnull().sum())

여행객.csv의 결측치 : 
TRAVELER_ID                  0
GENDER                       0
AGE_GRP                      0
JOB_NM                       0
TRAVEL_TERM                  0
TRAVEL_LIKE_SGG_1            0
TRAVEL_LIKE_SGG_2            0
TRAVEL_LIKE_SGG_3            0
TRAVEL_STYL_1                0
TRAVEL_STYL_2                0
TRAVEL_STYL_3                0
TRAVEL_STYL_4                0
TRAVEL_STYL_5                0
TRAVEL_STYL_6                0
TRAVEL_STYL_7                0
TRAVEL_STYL_8                0
TRAVEL_STATUS_RESIDENCE      0
TRAVEL_STATUS_DESTINATION    0
TRAVEL_STATUS_ACCOMPANY      0
TRAVEL_MOTIVE                0
TRAVEL_COMPANIONS_NUM        0
dtype: int64


In [1601]:
# 데이터 살펴보기
traveller_master.head()

,TRAVELER_ID,GENDER,AGE_GRP,JOB_NM,TRAVEL_TERM,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE,TRAVEL_COMPANIONS_NUM
0,g002178,여,20,12,3,50130,50110,26500,1,1,1,3,3,2,6,6,광주광역시,전남,2인 여행(가족 외),1,1
1,g004485,남,40,2,2,48840,48220,26500,1,1,3,6,4,2,7,7,대전광역시,전북,2인 여행(가족 외),1,1
2,g005167,남,20,1,3,46710,45190,47170,2,3,1,4,6,3,1,7,서울특별시,전남,2인 여행(가족 외),7,1
3,g002887,여,30,3,2,42150,42210,28710,2,2,3,2,3,5,2,6,인천광역시,대전,2인 여행(가족 외),2,1
4,g004189,남,20,4,2,42150,50110,26350,2,1,2,4,6,5,6,5,경기도,충남,2인 여행(가족 외),2,1


<h3>2. 전처리</h3>

<h5>1) 모든 CSV 파일 통합을 위해 TRAVELER_ID를 TRAVEL_ID 형식으로 변환</h5>

In [1602]:
# TRAVELER_ID의 가장 앞의 알파벳과 _를 추출하여 TRAVELER_ID에 결합하기 위한 함수
def modify_travel_id(travel_id):
    match = re.match(r'([a-zA-Z])(\d+)', travel_id)
    if match:
        return f'{match.group(1)}_{match.group(0)}'
    return travel_id

# TRAVELER_ID 변환
traveller_master['TRAVEL_ID'] = traveller_master['TRAVELER_ID'].apply(modify_travel_id)

# 변환 결과 확인
traveller_master[['TRAVELER_ID', 'TRAVEL_ID']].head()

,TRAVELER_ID,TRAVEL_ID
0,g002178,g_g002178
1,g004485,g_g004485
2,g005167,g_g005167
3,g002887,g_g002887
4,g004189,g_g004189


In [1603]:
# TRAVELER_ID 컬럼 삭제
traveller_master.drop(columns=['TRAVELER_ID'], inplace=True)

# 결과 확인
traveller_master.head()

,GENDER,AGE_GRP,JOB_NM,TRAVEL_TERM,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE,TRAVEL_COMPANIONS_NUM,TRAVEL_ID
0,여,20,12,3,50130,50110,26500,1,1,1,3,3,2,6,6,광주광역시,전남,2인 여행(가족 외),1,1,g_g002178
1,남,40,2,2,48840,48220,26500,1,1,3,6,4,2,7,7,대전광역시,전북,2인 여행(가족 외),1,1,g_g004485
2,남,20,1,3,46710,45190,47170,2,3,1,4,6,3,1,7,서울특별시,전남,2인 여행(가족 외),7,1,g_g005167
3,여,30,3,2,42150,42210,28710,2,2,3,2,3,5,2,6,인천광역시,대전,2인 여행(가족 외),2,1,g_g002887
4,남,20,4,2,42150,50110,26350,2,1,2,4,6,5,6,5,경기도,충남,2인 여행(가족 외),2,1,g_g004189


<h5>2) JOB_NM 변수 코드 'JOB' 매핑</h5>

In [1604]:
traveller_master = convert_codes_to_korean(traveller_master, 'JOB_NM', JOB_mapping)

# 결과 확인
traveller_master.head()

,GENDER,AGE_GRP,JOB_NM,TRAVEL_TERM,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE,TRAVEL_COMPANIONS_NUM,TRAVEL_ID
0,여,20,학생,3,50130,50110,26500,1,1,1,3,3,2,6,6,광주광역시,전남,2인 여행(가족 외),1,1,g_g002178
1,남,40,전문가 및 관련 종사자,2,48840,48220,26500,1,1,3,6,4,2,7,7,대전광역시,전북,2인 여행(가족 외),1,1,g_g004485
2,남,20,관리자,3,46710,45190,47170,2,3,1,4,6,3,1,7,서울특별시,전남,2인 여행(가족 외),7,1,g_g005167
3,여,30,사무 종사자,2,42150,42210,28710,2,2,3,2,3,5,2,6,인천광역시,대전,2인 여행(가족 외),2,1,g_g002887
4,남,20,서비스 종사자,2,42150,50110,26350,2,1,2,4,6,5,6,5,경기도,충남,2인 여행(가족 외),2,1,g_g004189


<h5>3) TRAVEL_TERM 변수 코드 'TTM' 매핑</h5>

In [1605]:
traveller_master = convert_codes_to_korean(traveller_master, 'TRAVEL_TERM', TTM_mapping)

# 결과 확인
traveller_master.head()

,GENDER,AGE_GRP,JOB_NM,TRAVEL_TERM,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE,TRAVEL_COMPANIONS_NUM,TRAVEL_ID
0,여,20,학생,1년,50130,50110,26500,1,1,1,3,3,2,6,6,광주광역시,전남,2인 여행(가족 외),1,1,g_g002178
1,남,40,전문가 및 관련 종사자,한달,48840,48220,26500,1,1,3,6,4,2,7,7,대전광역시,전북,2인 여행(가족 외),1,1,g_g004485
2,남,20,관리자,1년,46710,45190,47170,2,3,1,4,6,3,1,7,서울특별시,전남,2인 여행(가족 외),7,1,g_g005167
3,여,30,사무 종사자,한달,42150,42210,28710,2,2,3,2,3,5,2,6,인천광역시,대전,2인 여행(가족 외),2,1,g_g002887
4,남,20,서비스 종사자,한달,42150,50110,26350,2,1,2,4,6,5,6,5,경기도,충남,2인 여행(가족 외),2,1,g_g004189


<h5>4) TRAVEL_STYL_1~8 변수 코드 'TSY' 매핑</h5>

In [1606]:
traveller_master = convert_codes_to_korean(traveller_master, 'TRAVEL_STYL_1', TSY_mapping)
traveller_master = convert_codes_to_korean(traveller_master, 'TRAVEL_STYL_2', TSY_mapping)
traveller_master = convert_codes_to_korean(traveller_master, 'TRAVEL_STYL_3', TSY_mapping)
traveller_master = convert_codes_to_korean(traveller_master, 'TRAVEL_STYL_4', TSY_mapping)
traveller_master = convert_codes_to_korean(traveller_master, 'TRAVEL_STYL_5', TSY_mapping)
traveller_master = convert_codes_to_korean(traveller_master, 'TRAVEL_STYL_6', TSY_mapping)
traveller_master = convert_codes_to_korean(traveller_master, 'TRAVEL_STYL_7', TSY_mapping)
traveller_master = convert_codes_to_korean(traveller_master, 'TRAVEL_STYL_8', TSY_mapping)

# 결과 확인
traveller_master.head()

,GENDER,AGE_GRP,JOB_NM,TRAVEL_TERM,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE,TRAVEL_COMPANIONS_NUM,TRAVEL_ID
0,여,20,학생,1년,50130,50110,26500,자연 선호(강),자연 선호(강),자연 선호(강),자연 선호(약),자연 선호(약),자연 선호(중),도시 선호(중),도시 선호(중),광주광역시,전남,2인 여행(가족 외),1,1,g_g002178
1,남,40,전문가 및 관련 종사자,한달,48840,48220,26500,자연 선호(강),자연 선호(강),자연 선호(약),도시 선호(중),중립,자연 선호(중),도시 선호(강),도시 선호(강),대전광역시,전북,2인 여행(가족 외),1,1,g_g004485
2,남,20,관리자,1년,46710,45190,47170,자연 선호(중),자연 선호(약),자연 선호(강),중립,도시 선호(중),자연 선호(약),자연 선호(강),도시 선호(강),서울특별시,전남,2인 여행(가족 외),7,1,g_g005167
3,여,30,사무 종사자,한달,42150,42210,28710,자연 선호(중),자연 선호(중),자연 선호(약),자연 선호(중),자연 선호(약),도시 선호(약),자연 선호(중),도시 선호(중),인천광역시,대전,2인 여행(가족 외),2,1,g_g002887
4,남,20,서비스 종사자,한달,42150,50110,26350,자연 선호(중),자연 선호(강),자연 선호(중),중립,도시 선호(중),도시 선호(약),도시 선호(중),도시 선호(약),경기도,충남,2인 여행(가족 외),2,1,g_g004189


<h5>5) TRAVEL_MOTIVE 변수 코드 'TMT' 매핑</h5>

In [1607]:
traveller_master = convert_codes_to_korean(traveller_master, 'TRAVEL_MOTIVE', TMT_mapping)

# 결과 확인
traveller_master.head()

,GENDER,AGE_GRP,JOB_NM,TRAVEL_TERM,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE,TRAVEL_COMPANIONS_NUM,TRAVEL_ID
0,여,20,학생,1년,50130,50110,26500,자연 선호(강),자연 선호(강),자연 선호(강),자연 선호(약),자연 선호(약),자연 선호(중),도시 선호(중),도시 선호(중),광주광역시,전남,2인 여행(가족 외),"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",1,g_g002178
1,남,40,전문가 및 관련 종사자,한달,48840,48220,26500,자연 선호(강),자연 선호(강),자연 선호(약),도시 선호(중),중립,자연 선호(중),도시 선호(강),도시 선호(강),대전광역시,전북,2인 여행(가족 외),"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",1,g_g004485
2,남,20,관리자,1년,46710,45190,47170,자연 선호(중),자연 선호(약),자연 선호(강),중립,도시 선호(중),자연 선호(약),자연 선호(강),도시 선호(강),서울특별시,전남,2인 여행(가족 외),새로운 경험 추구,1,g_g005167
3,여,30,사무 종사자,한달,42150,42210,28710,자연 선호(중),자연 선호(중),자연 선호(약),자연 선호(중),자연 선호(약),도시 선호(약),자연 선호(중),도시 선호(중),인천광역시,대전,2인 여행(가족 외),"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",1,g_g002887
4,남,20,서비스 종사자,한달,42150,50110,26350,자연 선호(중),자연 선호(강),자연 선호(중),중립,도시 선호(중),도시 선호(약),도시 선호(중),도시 선호(약),경기도,충남,2인 여행(가족 외),"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",1,g_g004189


<h3>3. 전처리 후 확인</h3>

In [1608]:
# TRAVEL_ID 중복 여부 확인
duplicates = traveller_master[traveller_master.duplicated(subset='TRAVEL_ID', keep=False)]

# 중복된 항목이 있는 경우
if not duplicates.empty:
    print("중복된 TRAVEL_ID가 있습니다.")
    print(duplicates)
else:
    print("중복된 TRAVEL_ID가 없습니다.")

중복된 TRAVEL_ID가 없습니다.


In [1609]:
# 데이터의 구조와 타입 확인
print('여행객.csv의 구조와 타입 : ')
print(traveller_master.info())

여행객.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2560 entries, 0 to 2559
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   GENDER                     2560 non-null   object
 1   AGE_GRP                    2560 non-null   int64 
 2   JOB_NM                     2560 non-null   object
 3   TRAVEL_TERM                2560 non-null   object
 4   TRAVEL_LIKE_SGG_1          2560 non-null   int64 
 5   TRAVEL_LIKE_SGG_2          2560 non-null   int64 
 6   TRAVEL_LIKE_SGG_3          2560 non-null   int64 
 7   TRAVEL_STYL_1              2560 non-null   object
 8   TRAVEL_STYL_2              2560 non-null   object
 9   TRAVEL_STYL_3              2560 non-null   object
 10  TRAVEL_STYL_4              2560 non-null   object
 11  TRAVEL_STYL_5              2560 non-null   object
 12  TRAVEL_STYL_6              2560 non-null   object
 13  TRAVEL_STYL_7              2560 non-null   o

In [1610]:
# 데이터의 결측치 확인
print('여행객.csv의 결측치 : ')
print(traveller_master.isnull().sum())

여행객.csv의 결측치 : 
GENDER                       0
AGE_GRP                      0
JOB_NM                       0
TRAVEL_TERM                  0
TRAVEL_LIKE_SGG_1            0
TRAVEL_LIKE_SGG_2            0
TRAVEL_LIKE_SGG_3            0
TRAVEL_STYL_1                0
TRAVEL_STYL_2                0
TRAVEL_STYL_3                0
TRAVEL_STYL_4                0
TRAVEL_STYL_5                0
TRAVEL_STYL_6                0
TRAVEL_STYL_7                0
TRAVEL_STYL_8                0
TRAVEL_STATUS_RESIDENCE      0
TRAVEL_STATUS_DESTINATION    0
TRAVEL_STATUS_ACCOMPANY      0
TRAVEL_MOTIVE                0
TRAVEL_COMPANIONS_NUM        0
TRAVEL_ID                    0
dtype: int64


In [1611]:
# 데이터 살펴보기
traveller_master.head()

,GENDER,AGE_GRP,JOB_NM,TRAVEL_TERM,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE,TRAVEL_COMPANIONS_NUM,TRAVEL_ID
0,여,20,학생,1년,50130,50110,26500,자연 선호(강),자연 선호(강),자연 선호(강),자연 선호(약),자연 선호(약),자연 선호(중),도시 선호(중),도시 선호(중),광주광역시,전남,2인 여행(가족 외),"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",1,g_g002178
1,남,40,전문가 및 관련 종사자,한달,48840,48220,26500,자연 선호(강),자연 선호(강),자연 선호(약),도시 선호(중),중립,자연 선호(중),도시 선호(강),도시 선호(강),대전광역시,전북,2인 여행(가족 외),"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",1,g_g004485
2,남,20,관리자,1년,46710,45190,47170,자연 선호(중),자연 선호(약),자연 선호(강),중립,도시 선호(중),자연 선호(약),자연 선호(강),도시 선호(강),서울특별시,전남,2인 여행(가족 외),새로운 경험 추구,1,g_g005167
3,여,30,사무 종사자,한달,42150,42210,28710,자연 선호(중),자연 선호(중),자연 선호(약),자연 선호(중),자연 선호(약),도시 선호(약),자연 선호(중),도시 선호(중),인천광역시,대전,2인 여행(가족 외),"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",1,g_g002887
4,남,20,서비스 종사자,한달,42150,50110,26350,자연 선호(중),자연 선호(강),자연 선호(중),중립,도시 선호(중),도시 선호(약),도시 선호(중),도시 선호(약),경기도,충남,2인 여행(가족 외),"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",1,g_g004189


<hr>
<h2>[동반자정보.csv]</h2>

<h3>1. 변수 추출 및 검토</h3>

In [1612]:
# 필요한 변수 리스트
columns_to_keep_companion = [
    'TRAVEL_ID', 'REL_CD', 'COMPANION_GENDER', 'COMPANION_AGE_GRP'
    ]

# companion_info에 필요한 변수만 남기고 저장
companion_info = companion_info[columns_to_keep_companion]

# 결과 확인
print(companion_info.head())

   TRAVEL_ID  REL_CD  COMPANION_GENDER  COMPANION_AGE_GRP
0  e_e000005       1                 1                  4
1  e_e000191       1                 2                  5
2  e_e000191      11                 2                  5
3  e_e000191      11                 2                  8
4  e_e000191      11                 1                  6


In [1613]:
# 데이터의 행, 열 개수 출력
print('동반자정보.csv의 행, 열 개수 : ')
print(companion_info.shape)

동반자정보.csv의 행, 열 개수 : 
(3808, 4)


In [1614]:
# 데이터의 구조와 타입 확인
print('동반자정보.csv의 구조와 타입 : ')
print(companion_info.info())

동반자정보.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3808 entries, 0 to 3807
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   TRAVEL_ID          3808 non-null   object
 1   REL_CD             3808 non-null   int64 
 2   COMPANION_GENDER   3808 non-null   int64 
 3   COMPANION_AGE_GRP  3808 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 119.1+ KB
None


In [1615]:
# 데이터의 결측치 확인
print('동반자정보.csv의 결측치 : ')
print(companion_info.isnull().sum())

동반자정보.csv의 결측치 : 
TRAVEL_ID            0
REL_CD               0
COMPANION_GENDER     0
COMPANION_AGE_GRP    0
dtype: int64


In [1616]:
# 데이터 살펴보기
companion_info.head()

,TRAVEL_ID,REL_CD,COMPANION_GENDER,COMPANION_AGE_GRP
0,e_e000005,1,1,4
1,e_e000191,1,2,5
2,e_e000191,11,2,5
3,e_e000191,11,2,8
4,e_e000191,11,1,6


<h3>2. 전처리</h3>

<h5>1) REL_CD 변수 코드 'TCR' 매핑</h5>

In [1617]:
companion_info = convert_codes_to_korean(companion_info, 'REL_CD', TCR_mapping)

# 결과 확인
companion_info.head()

,TRAVEL_ID,REL_CD,COMPANION_GENDER,COMPANION_AGE_GRP
0,e_e000005,배우자,1,4
1,e_e000191,배우자,2,5
2,e_e000191,기타,2,5
3,e_e000191,기타,2,8
4,e_e000191,기타,1,6


<h5>2) COMPANION_GENDER 변수 코드 'GEN' 매핑</h5>

In [1618]:
companion_info = convert_codes_to_korean(companion_info, 'COMPANION_GENDER', GEN_mapping)

# 결과 확인
companion_info.head()

,TRAVEL_ID,REL_CD,COMPANION_GENDER,COMPANION_AGE_GRP
0,e_e000005,배우자,남자,4
1,e_e000191,배우자,여자,5
2,e_e000191,기타,여자,5
3,e_e000191,기타,여자,8
4,e_e000191,기타,남자,6


<h5>3) COMPANION_AGE_GRP 변수 코드 'AGE' 매핑</h5>

In [1619]:
companion_info = convert_codes_to_korean(companion_info, 'COMPANION_AGE_GRP', AGE_mapping)

# 결과 확인
companion_info.head()

,TRAVEL_ID,REL_CD,COMPANION_GENDER,COMPANION_AGE_GRP
0,e_e000005,배우자,남자,30대
1,e_e000191,배우자,여자,40대
2,e_e000191,기타,여자,40대
3,e_e000191,기타,여자,70세이상
4,e_e000191,기타,남자,50대


<h3>3. 전처리 후 확인</h3>

In [1620]:
# TRAVEL_ID 중복 여부 확인
duplicates = companion_info[companion_info.duplicated(subset='TRAVEL_ID', keep=False)]

# 중복된 항목이 있는 경우
if not duplicates.empty:
    print("중복된 TRAVEL_ID가 있습니다.")
    print(duplicates)
else:
    print("중복된 TRAVEL_ID가 없습니다.")

중복된 TRAVEL_ID가 있습니다.
      TRAVEL_ID REL_CD COMPANION_GENDER COMPANION_AGE_GRP
1     e_e000191    배우자               여자               40대
2     e_e000191     기타               여자               40대
3     e_e000191     기타               여자             70세이상
4     e_e000191     기타               남자               50대
6     e_e000668     자녀               남자               10대
...         ...    ...              ...               ...
3795  g_g013084     부모               남자               60대
3796  g_g013084     부모               여자               60대
3797  g_g013087  형제/자매               여자               20대
3798  g_g013087     부모               여자               50대
3799  g_g013087  형제/자매               여자               10대

[2609 rows x 4 columns]


In [1621]:
# 데이터의 구조와 타입 확인
print('동반자정보.csv의 구조와 타입 : ')
print(companion_info.info())

동반자정보.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3808 entries, 0 to 3807
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   TRAVEL_ID          3808 non-null   object
 1   REL_CD             3808 non-null   object
 2   COMPANION_GENDER   3808 non-null   object
 3   COMPANION_AGE_GRP  3808 non-null   object
dtypes: object(4)
memory usage: 119.1+ KB
None


In [1622]:
# 데이터의 결측치 확인
print('동반자정보.csv의 결측치 : ')
print(companion_info.isnull().sum())

동반자정보.csv의 결측치 : 
TRAVEL_ID            0
REL_CD               0
COMPANION_GENDER     0
COMPANION_AGE_GRP    0
dtype: int64


In [1623]:
# 데이터 살펴보기
companion_info.head()

,TRAVEL_ID,REL_CD,COMPANION_GENDER,COMPANION_AGE_GRP
0,e_e000005,배우자,남자,30대
1,e_e000191,배우자,여자,40대
2,e_e000191,기타,여자,40대
3,e_e000191,기타,여자,70세이상
4,e_e000191,기타,남자,50대


<hr>
<h2>[여행.csv]</h2>

<h3>1. 변수 추출 및 검토</h3>

In [1624]:
# 필요한 변수 리스트
columns_to_keep_travel = [
    'TRAVEL_ID', 'TRAVEL_PURPOSE', 'TRAVEL_START_YMD', 'TRAVEL_END_YMD', 'TRAVEL_MISSION_CHECK'
    ]

# travel에 필요한 변수만 남기고 저장
travel = travel[columns_to_keep_travel]

# 결과 확인
print(travel.head())

   TRAVEL_ID TRAVEL_PURPOSE TRAVEL_START_YMD TRAVEL_END_YMD TRAVEL_MISSION_CHECK
0  e_e000005         5;2;23       2023-04-29     2023-05-03               5;2;23
1  e_e000043    10;12;21;27       2023-04-28     2023-05-01                4;6;2
2  e_e000191             21       2023-04-28     2023-05-01             24;13;27
3  e_e000316  21;22;24;26;6       2023-05-27     2023-05-30              22;26;6
4  e_e000668              2       2023-05-20     2023-05-21                2;1;4


In [1625]:
# 데이터의 행, 열 개수 출력
print('여행.csv의 행, 열 개수 : ')
print(travel.shape)

여행.csv의 행, 열 개수 : 
(2560, 5)


In [1626]:
# 데이터의 구조와 타입 확인
print('여행.csv의 구조와 타입 : ')
print(travel.info())

여행.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2560 entries, 0 to 2559
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   TRAVEL_ID             2560 non-null   object
 1   TRAVEL_PURPOSE        2560 non-null   object
 2   TRAVEL_START_YMD      2560 non-null   object
 3   TRAVEL_END_YMD        2560 non-null   object
 4   TRAVEL_MISSION_CHECK  2560 non-null   object
dtypes: object(5)
memory usage: 100.1+ KB
None


In [1627]:
# 데이터의 결측치 확인
print('여행.csv의 결측치 : ')
print(travel.isnull().sum())

여행.csv의 결측치 : 
TRAVEL_ID               0
TRAVEL_PURPOSE          0
TRAVEL_START_YMD        0
TRAVEL_END_YMD          0
TRAVEL_MISSION_CHECK    0
dtype: int64


In [1628]:
# 데이터 살펴보기
travel.head()

,TRAVEL_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,TRAVEL_MISSION_CHECK
0,e_e000005,5;2;23,2023-04-29,2023-05-03,5;2;23
1,e_e000043,10;12;21;27,2023-04-28,2023-05-01,4;6;2
2,e_e000191,21,2023-04-28,2023-05-01,24;13;27
3,e_e000316,21;22;24;26;6,2023-05-27,2023-05-30,22;26;6
4,e_e000668,2,2023-05-20,2023-05-21,2;1;4


<h3>2. 전처리</h3>

<h5>1) TRAVEL_PURPOSE 변수 코드 'MIS' 매핑</h5>

In [1629]:
# TRAVEL_PURPOSE 변수의 값이 세미콜론으로 분리되어 있으므로, 세미콜론 기준으로 분리하고 매핑 적용하는 함수
def convert_colon_to_korean(df, column_name, mapping):
    if column_name in df.columns:
        # 세미콜론으로 분리된 값들을 매핑 처리
        df[column_name] = df[column_name].apply(
            lambda x: ';'.join([mapping.get(int(i), str(i)) for i in x.split(';')])
        )
    return df

In [1630]:
travel = convert_colon_to_korean(travel, 'TRAVEL_PURPOSE', MIS_mapping)

# 결과 확인
travel.head()

,TRAVEL_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,TRAVEL_MISSION_CHECK
0,e_e000005,"야외 스포츠, 레포츠 활동;테마파크, 놀이시설, 동/식물원 방문;호캉스 여행",2023-04-29,2023-05-03,5;2;23
1,e_e000043,온천/스파;드라마 촬영지 방문;Well-ness 여행;친환경 여행(플로깅 여행),2023-04-28,2023-05-01,4;6;2
2,e_e000191,Well-ness 여행,2023-04-28,2023-05-01,24;13;27
3,e_e000316,Well-ness 여행;SNS 인생샷 여행;신규 여행지 발굴;인플루언서 따라하기 여...,2023-05-27,2023-05-30,22;26;6
4,e_e000668,"테마파크, 놀이시설, 동/식물원 방문",2023-05-20,2023-05-21,2;1;4


<h5>2) TRAVEL_MISSION_CHECK 변수 코드 'MIS' 매핑</h5>

In [1631]:
travel = convert_colon_to_korean(travel, 'TRAVEL_MISSION_CHECK', MIS_mapping)

# 결과 확인
travel.head()

,TRAVEL_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,TRAVEL_MISSION_CHECK
0,e_e000005,"야외 스포츠, 레포츠 활동;테마파크, 놀이시설, 동/식물원 방문;호캉스 여행",2023-04-29,2023-05-03,"야외 스포츠, 레포츠 활동;테마파크, 놀이시설, 동/식물원 방문;호캉스 여행"
1,e_e000043,온천/스파;드라마 촬영지 방문;Well-ness 여행;친환경 여행(플로깅 여행),2023-04-28,2023-05-01,"시티투어;지역 문화예술/공연/전시시설 관람;테마파크, 놀이시설, 동/식물원 방문"
2,e_e000191,Well-ness 여행,2023-04-28,2023-05-01,신규 여행지 발굴;종교/성지 순례;친환경 여행(플로깅 여행)
3,e_e000316,Well-ness 여행;SNS 인생샷 여행;신규 여행지 발굴;인플루언서 따라하기 여...,2023-05-27,2023-05-30,SNS 인생샷 여행;인플루언서 따라하기 여행;지역 문화예술/공연/전시시설 관람
4,e_e000668,"테마파크, 놀이시설, 동/식물원 방문",2023-05-20,2023-05-21,"테마파크, 놀이시설, 동/식물원 방문;쇼핑;시티투어"


<h3>3. 전처리 후 확인</h3>

In [1632]:
# TRAVEL_ID 중복 여부 확인
duplicates = travel[travel.duplicated(subset='TRAVEL_ID', keep=False)]

# 중복된 항목이 있는 경우
if not duplicates.empty:
    print("중복된 TRAVEL_ID가 있습니다.")
    print(duplicates)
else:
    print("중복된 TRAVEL_ID가 없습니다.")

중복된 TRAVEL_ID가 없습니다.


In [1633]:
# 데이터의 구조와 타입 확인
print('여행.csv의 구조와 타입 : ')
print(travel.info())

여행.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2560 entries, 0 to 2559
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   TRAVEL_ID             2560 non-null   object
 1   TRAVEL_PURPOSE        2560 non-null   object
 2   TRAVEL_START_YMD      2560 non-null   object
 3   TRAVEL_END_YMD        2560 non-null   object
 4   TRAVEL_MISSION_CHECK  2560 non-null   object
dtypes: object(5)
memory usage: 100.1+ KB
None


In [1634]:
# 데이터의 결측치 확인
print('여행.csv의 결측치 : ')
print(travel.isnull().sum())

여행.csv의 결측치 : 
TRAVEL_ID               0
TRAVEL_PURPOSE          0
TRAVEL_START_YMD        0
TRAVEL_END_YMD          0
TRAVEL_MISSION_CHECK    0
dtype: int64


In [1635]:
# 데이터 살펴보기
travel.head()

,TRAVEL_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,TRAVEL_MISSION_CHECK
0,e_e000005,"야외 스포츠, 레포츠 활동;테마파크, 놀이시설, 동/식물원 방문;호캉스 여행",2023-04-29,2023-05-03,"야외 스포츠, 레포츠 활동;테마파크, 놀이시설, 동/식물원 방문;호캉스 여행"
1,e_e000043,온천/스파;드라마 촬영지 방문;Well-ness 여행;친환경 여행(플로깅 여행),2023-04-28,2023-05-01,"시티투어;지역 문화예술/공연/전시시설 관람;테마파크, 놀이시설, 동/식물원 방문"
2,e_e000191,Well-ness 여행,2023-04-28,2023-05-01,신규 여행지 발굴;종교/성지 순례;친환경 여행(플로깅 여행)
3,e_e000316,Well-ness 여행;SNS 인생샷 여행;신규 여행지 발굴;인플루언서 따라하기 여...,2023-05-27,2023-05-30,SNS 인생샷 여행;인플루언서 따라하기 여행;지역 문화예술/공연/전시시설 관람
4,e_e000668,"테마파크, 놀이시설, 동/식물원 방문",2023-05-20,2023-05-21,"테마파크, 놀이시설, 동/식물원 방문;쇼핑;시티투어"


<hr>
<h2>[이동수단소비내역.csv]</h2>

In [1636]:
# 필요한 변수 리스트
columns_to_keep_mvmn_consume = [
    'TRAVEL_ID', 'MVMN_SE_NM', 'PAYMENT_AMT_WON'
    ]

# mvmn_consume에 필요한 변수만 남기고 저장
mvmn_consume = mvmn_consume[columns_to_keep_mvmn_consume]

# 결과 확인
print(mvmn_consume.head())

   TRAVEL_ID        MVMN_SE_NM  PAYMENT_AMT_WON
0  g_g007947  렌터카(승용/승합/버스 등등)             1500
1  g_g000552                기타             1500
2  g_g000419           시외/고속버스             2000
3  g_g010747                택시             2000
4  g_g007226           시내/마을버스             2500


<h3>1. 변수 추출 및 검토</h3>

In [1637]:
# 데이터의 행, 열 개수 출력
print('이동수단소비내역.csv의 행, 열 개수 : ')
print(mvmn_consume.shape)

이동수단소비내역.csv의 행, 열 개수 : 
(5419, 3)


In [1638]:
# 데이터의 구조와 타입 확인
print('이동수단소비내역.csv의 구조와 타입 : ')
print(mvmn_consume.info())

이동수단소비내역.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5419 entries, 0 to 5418
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   TRAVEL_ID        5419 non-null   object
 1   MVMN_SE_NM       5419 non-null   object
 2   PAYMENT_AMT_WON  5419 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 127.1+ KB
None


In [1639]:
# 데이터의 결측치 확인
print('이동수단소비내역.csv의 결측치 : ')
print(mvmn_consume.isnull().sum())

이동수단소비내역.csv의 결측치 : 
TRAVEL_ID          0
MVMN_SE_NM         0
PAYMENT_AMT_WON    0
dtype: int64


In [1640]:
# 데이터 살펴보기
mvmn_consume.head()

,TRAVEL_ID,MVMN_SE_NM,PAYMENT_AMT_WON
0,g_g007947,렌터카(승용/승합/버스 등등),1500
1,g_g000552,기타,1500
2,g_g000419,시외/고속버스,2000
3,g_g010747,택시,2000
4,g_g007226,시내/마을버스,2500


<h3>2. 전처리</h3>

<h5>1) 이동수단 소비내역인 걸 나타내기 위해서 변수명 변경

In [1641]:
mvmn_consume = mvmn_consume.rename(columns = {'PAYMENT_AMT_WON' : 'MVMN_WON'})

# 결과 확인
mvmn_consume.head()

,TRAVEL_ID,MVMN_SE_NM,MVMN_WON
0,g_g007947,렌터카(승용/승합/버스 등등),1500
1,g_g000552,기타,1500
2,g_g000419,시외/고속버스,2000
3,g_g010747,택시,2000
4,g_g007226,시내/마을버스,2500


<h5>2) 추가적인 분석에 사용하기 위해 TRAVEL_ID별 주요이동수단 구하기

In [1642]:
# TRAVEL_ID별로 이동수단 빈도 계산
mvmn_consume_count = mvmn_consume.groupby(['TRAVEL_ID', 'MVMN_SE_NM']).size().reset_index(name='COUNT')

# TRAVEL_ID별로 가장 많이 사용된 이동수단 추출
mvmn_consume_major_transport = mvmn_consume_count.groupby('TRAVEL_ID').apply(
    lambda x: x[x['COUNT'] == x['COUNT'].max()]['MVMN_SE_NM'].tolist()  # 빈도수가 동일한 이동수단을 리스트로 결합
).reset_index(name='MAJOR_TRANSPORT')

# 결과 확인
print(mvmn_consume_major_transport.head())

   TRAVEL_ID     MAJOR_TRANSPORT
0  e_e000005  [자가용(승용/승합/트럭 등등)]
1  e_e000043  [자가용(승용/승합/트럭 등등)]
2  e_e000191  [자가용(승용/승합/트럭 등등)]
3  e_e000316                [택시]
4  e_e000668  [자가용(승용/승합/트럭 등등)]


<h5>3) TRAVEL_ID가 중복되는 경우 이동수단 소비내역 전처리 - pivot_table

In [1643]:
# TRAVEL_ID 중복 여부 확인
duplicates = mvmn_consume[mvmn_consume.duplicated(subset='TRAVEL_ID', keep=False)]

# 중복된 항목이 있는 경우
if not duplicates.empty:
    print("중복된 TRAVEL_ID가 있습니다.")
else:
    print("중복된 TRAVEL_ID가 없습니다.")

중복된 TRAVEL_ID가 있습니다.


pivot_table 형식으로 변환하여, 이후 파생변수 추가 등 추가적인 분석을 쉽게 할 수 있도록 함

In [1644]:
# MVMN_SE_NM을 문자열 형식으로 변환
mvmn_consume['MVMN_SE_NM'] = mvmn_consume['MVMN_SE_NM'].astype(str)

# pivot_table 생성
mvmn_consume_pivot = mvmn_consume.pivot_table(
    index='TRAVEL_ID', 
    columns='MVMN_SE_NM', 
    values='MVMN_WON', 
    aggfunc='sum', 
    fill_value=0  # 결측값은 0으로
)

# 결과 확인
print(mvmn_consume_pivot.head())

MVMN_SE_NM  KTX/SRT(고속열차)  관광버스  기타  렌터카(승용/승합/버스 등등)  배/선박  버스 + 지하철  새마을/무궁화열차  시내/마을버스  시외/고속버스  자가용(승용/승합/트럭 등등)  자전거  지하철  캠핑카(자차 및 렌탈)     택시  항공기
TRAVEL_ID                                                                                                                                               
e_e000005               0     0   0                 0     0         0          0        0        0            112300    0    0             0      0    0
e_e000043               0     0   0                 0     0         0          0        0        0             42925    0    0             0      0    0
e_e000191               0     0   0                 0     0         0          0        0        0             98000    0    0             0      0    0
e_e000316          140000     0   0                 0     0         0          0    40000        0              9000    0    0             0  15300    0
e_e000668               0     0   0                 0     0         0          0  

2)에서 구했던 주요이동수단과 병합하여 mvmn_consume_new 데이터프레임 생성

In [1645]:
# 주요 이동수단과 피벗 테이블 병합
mvmn_consume_new = pd.merge(
    mvmn_consume_pivot.reset_index(),  # 피벗 테이블을 병합하기 위해 인덱스를 초기화
    mvmn_consume_major_transport,     # 주요 이동수단 데이터프레임
    on='TRAVEL_ID',                   # 공통 키: TRAVEL_ID
    how='left'                        # 병합 방식: 왼쪽 (피벗 테이블 기준)
)

# 결과 확인
print(mvmn_consume_new.head())

   TRAVEL_ID  KTX/SRT(고속열차)  관광버스  기타  렌터카(승용/승합/버스 등등)  배/선박  버스 + 지하철  새마을/무궁화열차  시내/마을버스  시외/고속버스  자가용(승용/승합/트럭 등등)  자전거  지하철  캠핑카(자차 및 렌탈)     택시  항공기     MAJOR_TRANSPORT
0  e_e000005              0     0   0                 0     0         0          0        0        0            112300    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]
1  e_e000043              0     0   0                 0     0         0          0        0        0             42925    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]
2  e_e000191              0     0   0                 0     0         0          0        0        0             98000    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]
3  e_e000316         140000     0   0                 0     0         0          0    40000        0              9000    0    0             0  15300    0                [택시]
4  e_e000668              0     0   0                 0     0         0          0        0        0             86400    0  

<h5>4) 추가적인 분석에 사용하기 위해 TRAVEL_ID별 이동수단총소비액 파생변수 추가

In [1646]:
# TRAVEL_ID별로 이동수단의 총 소비액 계산
mvmn_consume_total = mvmn_consume.groupby('TRAVEL_ID')['MVMN_WON'].sum().reset_index(name='TOTAL_MVMN_WON')

# mvmn_consume_new와 병합
mvmn_consume_new = mvmn_consume_new.merge(mvmn_consume_total, on='TRAVEL_ID', how='left')

# 결과 확인
print(mvmn_consume_new.head())

   TRAVEL_ID  KTX/SRT(고속열차)  관광버스  기타  렌터카(승용/승합/버스 등등)  배/선박  버스 + 지하철  새마을/무궁화열차  시내/마을버스  시외/고속버스  자가용(승용/승합/트럭 등등)  자전거  지하철  캠핑카(자차 및 렌탈)     택시  항공기     MAJOR_TRANSPORT  TOTAL_MVMN_WON
0  e_e000005              0     0   0                 0     0         0          0        0        0            112300    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]          112300
1  e_e000043              0     0   0                 0     0         0          0        0        0             42925    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]           42925
2  e_e000191              0     0   0                 0     0         0          0        0        0             98000    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]           98000
3  e_e000316         140000     0   0                 0     0         0          0    40000        0              9000    0    0             0  15300    0                [택시]          204300
4  e_e000668              0     0   0        

<h3>3. 전처리 후 확인</h3>

In [1647]:
# TRAVEL_ID 중복 여부 확인
duplicates = mvmn_consume_new[mvmn_consume_new.duplicated(subset='TRAVEL_ID', keep=False)]

# 중복된 항목이 있는 경우
if not duplicates.empty:
    print("중복된 TRAVEL_ID가 있습니다.")
else:
    print("중복된 TRAVEL_ID가 없습니다.")

중복된 TRAVEL_ID가 없습니다.


In [1648]:
# 데이터의 구조와 타입 확인
print('이동수단소비내역.csv의 구조와 타입 : ')
print(mvmn_consume_new.info())

이동수단소비내역.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2370 entries, 0 to 2369
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   TRAVEL_ID         2370 non-null   object
 1   KTX/SRT(고속열차)     2370 non-null   int64 
 2   관광버스              2370 non-null   int64 
 3   기타                2370 non-null   int64 
 4   렌터카(승용/승합/버스 등등)  2370 non-null   int64 
 5   배/선박              2370 non-null   int64 
 6   버스 + 지하철          2370 non-null   int64 
 7   새마을/무궁화열차         2370 non-null   int64 
 8   시내/마을버스           2370 non-null   int64 
 9   시외/고속버스           2370 non-null   int64 
 10  자가용(승용/승합/트럭 등등)  2370 non-null   int64 
 11  자전거               2370 non-null   int64 
 12  지하철               2370 non-null   int64 
 13  캠핑카(자차 및 렌탈)      2370 non-null   int64 
 14  택시                2370 non-null   int64 
 15  항공기               2370 non-null   int64 
 16  MAJOR_TRANSPORT   2370 non-null   ob

In [1649]:
# 데이터의 결측치 확인
print('이동수단소비내역.csv의 결측치 : ')
print(mvmn_consume_new.isnull().sum())

이동수단소비내역.csv의 결측치 : 
TRAVEL_ID           0
KTX/SRT(고속열차)       0
관광버스                0
기타                  0
렌터카(승용/승합/버스 등등)    0
배/선박                0
버스 + 지하철            0
새마을/무궁화열차           0
시내/마을버스             0
시외/고속버스             0
자가용(승용/승합/트럭 등등)    0
자전거                 0
지하철                 0
캠핑카(자차 및 렌탈)        0
택시                  0
항공기                 0
MAJOR_TRANSPORT     0
TOTAL_MVMN_WON      0
dtype: int64


In [1650]:
# 데이터 살펴보기
mvmn_consume_new.head()

,TRAVEL_ID,KTX/SRT(고속열차),관광버스,기타,렌터카(승용/승합/버스 등등),배/선박,버스 + 지하철,새마을/무궁화열차,시내/마을버스,시외/고속버스,자가용(승용/승합/트럭 등등),자전거,지하철,캠핑카(자차 및 렌탈),택시,항공기,MAJOR_TRANSPORT,TOTAL_MVMN_WON
0,e_e000005,0,0,0,0,0,0,0,0,0,112300,0,0,0,0,0,[자가용(승용/승합/트럭 등등)],112300
1,e_e000043,0,0,0,0,0,0,0,0,0,42925,0,0,0,0,0,[자가용(승용/승합/트럭 등등)],42925
2,e_e000191,0,0,0,0,0,0,0,0,0,98000,0,0,0,0,0,[자가용(승용/승합/트럭 등등)],98000
3,e_e000316,140000,0,0,0,0,0,0,40000,0,9000,0,0,0,15300,0,[택시],204300
4,e_e000668,0,0,0,0,0,0,0,0,0,86400,0,0,0,0,0,[자가용(승용/승합/트럭 등등)],86400


<hr>
<h2>[숙박소비내역.csv]</h2>

<h3>1. 변수 추출 및 검토</h3>

In [1651]:
# 필요한 변수 리스트
columns_to_keep_lodge_consume = [
    'TRAVEL_ID', 'LODGING_TYPE_CD', 'PAYMENT_AMT_WON'
    ]

# lodge_consume.shape에 필요한 변수만 남기고 저장
lodge_consume = lodge_consume[columns_to_keep_lodge_consume]

# 결과 확인
print(lodge_consume.head())

   TRAVEL_ID  LODGING_TYPE_CD  PAYMENT_AMT_WON
0  g_g000950                3           108000
1  g_g003956                9            66000
2  g_g005377                5           120000
3  g_g000638                7            32000
4  g_g000773                4            80000


In [1652]:
# 데이터의 행, 열 개수 출력
print('숙박소비내역.csv의 행, 열 개수 : ')
print(lodge_consume.shape)

숙박소비내역.csv의 행, 열 개수 : 
(1333, 3)


In [1653]:
# 데이터의 구조와 타입 확인
print('숙박소비내역.csv의 구조와 타입 : ')
print(lodge_consume.info())

숙박소비내역.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333 entries, 0 to 1332
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   TRAVEL_ID        1333 non-null   object
 1   LODGING_TYPE_CD  1333 non-null   int64 
 2   PAYMENT_AMT_WON  1333 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 31.4+ KB
None


In [1654]:
# 데이터의 결측치 확인
print('숙박소비내역.csv의 결측치 : ')
print(lodge_consume.isnull().sum())

숙박소비내역.csv의 결측치 : 
TRAVEL_ID          0
LODGING_TYPE_CD    0
PAYMENT_AMT_WON    0
dtype: int64


In [1655]:
# 데이터 살펴보기
lodge_consume.head()

,TRAVEL_ID,LODGING_TYPE_CD,PAYMENT_AMT_WON
0,g_g000950,3,108000
1,g_g003956,9,66000
2,g_g005377,5,120000
3,g_g000638,7,32000
4,g_g000773,4,80000


<h3>2. 전처리</h3>

<h5>1) LODGING_TYPE_CD 변수 코드 'HTY' 매핑</h5>

In [1656]:
lodge_consume = convert_codes_to_korean(lodge_consume, 'LODGING_TYPE_CD', HTY_mapping)

# 결과 확인
lodge_consume.head()

,TRAVEL_ID,LODGING_TYPE_CD,PAYMENT_AMT_WON
0,g_g000950,펜션,108000
1,g_g003956,자연휴양림내 시설,66000
2,g_g005377,"야영(캠핑,글램핑등)",120000
3,g_g000638,게스트하우스,32000
4,g_g000773,콘도미니엄/리조트,80000


<h5>2) 숙박 소비내역인 걸 나타내기 위해서 변수명 변경</h5>

In [1657]:
lodge_consume = lodge_consume.rename(columns = {'PAYMENT_AMT_WON' : 'LODGE_WON'})

# 결과 확인
lodge_consume.head()

,TRAVEL_ID,LODGING_TYPE_CD,LODGE_WON
0,g_g000950,펜션,108000
1,g_g003956,자연휴양림내 시설,66000
2,g_g005377,"야영(캠핑,글램핑등)",120000
3,g_g000638,게스트하우스,32000
4,g_g000773,콘도미니엄/리조트,80000


<h5>3) 추가적인 분석에 사용하기 위해 TRAVEL_ID별 주요숙소유형 구하기

In [1658]:
# TRAVEL_ID별로 숙소유형 빈도 계산
lodging_count = lodge_consume.groupby(['TRAVEL_ID', 'LODGING_TYPE_CD']).size().reset_index(name='COUNT')

# TRAVEL_ID별로 가장 많이 사용된 숙소유형 추출
lodge_major_type = lodging_count.groupby('TRAVEL_ID').apply(
    lambda x: x[x['COUNT'] == x['COUNT'].max()]['LODGING_TYPE_CD'].tolist()  # 빈도수가 동일한 숙소 유형을 리스트로 결합
).reset_index(name='MAJOR_LODGE_TYPE')

# 결과 확인
print(lodge_major_type.head())

   TRAVEL_ID        MAJOR_LODGE_TYPE
0  e_e000005  [모텔/여관, 콘도미니엄/리조트, 호텔]
1  e_e000191                    [펜션]
2  e_e000316                 [모텔/여관]
3  e_e004870             [자연휴양림내 시설]
4  f_f000001           [야영(캠핑,글램핑등)]


<h5>4) TRAVEL_ID가 중복되는 경우 숙소 소비내역 전처리 - pivot_table

In [1659]:
# TRAVEL_ID 중복 여부 확인
duplicates = lodge_consume[lodge_consume.duplicated(subset='TRAVEL_ID', keep=False)]

# 중복된 항목이 있는 경우
if not duplicates.empty:
    print("중복된 TRAVEL_ID가 있습니다.")
else:
    print("중복된 TRAVEL_ID가 없습니다.")

중복된 TRAVEL_ID가 있습니다.


pivot_table 형식으로 변환하여, 이후 파생변수 추가 등 추가적인 분석을 쉽게 할 수 있도록 함

In [1660]:
# pivot_table 생성
lodge_consume_pivot = lodge_consume.pivot_table(
    index='TRAVEL_ID', 
    columns='LODGING_TYPE_CD', 
    values='LODGE_WON', 
    aggfunc='sum', 
    fill_value=0  # 결측값은 0으로
)

# 결과 확인
print(lodge_consume_pivot.head())

LODGING_TYPE_CD  가족/친지집  게스트하우스  기타   모텔/여관  민박  야영(캠핑,글램핑등)  자연휴양림내 시설  전통 숙박시설  콘도미니엄/리조트      펜션     호텔
TRAVEL_ID                                                                                                 
e_e000005             0       0   0   87000   0            0          0        0     121440       0  90000
e_e000191             0       0   0       0   0            0          0        0          0  240000      0
e_e000316             0       0   0  131095   0            0          0        0          0       0      0
e_e004870             0       0   0       0   0            0     260000        0          0       0      0
f_f000001             0       0   0       0   0       100000          0        0          0       0      0


3)에서 구했던 주요숙소유형과 병합하여 lodge_consume_new 데이터프레임 생성

In [1661]:
# 주요 이동수단과 피벗 테이블 병합
lodge_consume_new = pd.merge(
    lodge_consume_pivot.reset_index(),  # 피벗 테이블을 병합하기 위해 인덱스를 초기화
    lodge_major_type,     # 주요 이동수단 데이터프레임
    on='TRAVEL_ID',                   # 공통 키: TRAVEL_ID
    how='left'                        # 병합 방식: 왼쪽 (피벗 테이블 기준)
)

# 결과 확인
print(lodge_consume_new.head())

   TRAVEL_ID  가족/친지집  게스트하우스  기타   모텔/여관  민박  야영(캠핑,글램핑등)  자연휴양림내 시설  전통 숙박시설  콘도미니엄/리조트      펜션     호텔        MAJOR_LODGE_TYPE
0  e_e000005       0       0   0   87000   0            0          0        0     121440       0  90000  [모텔/여관, 콘도미니엄/리조트, 호텔]
1  e_e000191       0       0   0       0   0            0          0        0          0  240000      0                    [펜션]
2  e_e000316       0       0   0  131095   0            0          0        0          0       0      0                 [모텔/여관]
3  e_e004870       0       0   0       0   0            0     260000        0          0       0      0             [자연휴양림내 시설]
4  f_f000001       0       0   0       0   0       100000          0        0          0       0      0           [야영(캠핑,글램핑등)]


<h5>4) 추가적인 분석에 사용하기 위해 TRAVEL_ID별 숙소총소비액 파생변수 추가

In [1662]:
# TRAVEL_ID별로 숙소 총 소비액 계산
lodge_consume_total = lodge_consume.groupby('TRAVEL_ID')['LODGE_WON'].sum().reset_index(name='TOTAL_LODGE_WON')

# lodge_consume_new와 병합
lodge_consume_new = lodge_consume_new.merge(lodge_consume_total, on='TRAVEL_ID', how='left')

# 결과 확인
print(lodge_consume_new.head())

   TRAVEL_ID  가족/친지집  게스트하우스  기타   모텔/여관  민박  야영(캠핑,글램핑등)  자연휴양림내 시설  전통 숙박시설  콘도미니엄/리조트      펜션     호텔        MAJOR_LODGE_TYPE  TOTAL_LODGE_WON
0  e_e000005       0       0   0   87000   0            0          0        0     121440       0  90000  [모텔/여관, 콘도미니엄/리조트, 호텔]           298440
1  e_e000191       0       0   0       0   0            0          0        0          0  240000      0                    [펜션]           240000
2  e_e000316       0       0   0  131095   0            0          0        0          0       0      0                 [모텔/여관]           131095
3  e_e004870       0       0   0       0   0            0     260000        0          0       0      0             [자연휴양림내 시설]           260000
4  f_f000001       0       0   0       0   0       100000          0        0          0       0      0           [야영(캠핑,글램핑등)]           100000


<h3>3. 전처리 후 확인</h3>

In [1663]:
# TRAVEL_ID 중복 여부 확인
duplicates = lodge_consume_new[lodge_consume_new.duplicated(subset='TRAVEL_ID', keep=False)]

# 중복된 항목이 있는 경우
if not duplicates.empty:
    print("중복된 TRAVEL_ID가 있습니다.")
else:
    print("중복된 TRAVEL_ID가 없습니다.")

중복된 TRAVEL_ID가 없습니다.


In [1664]:
# 데이터의 구조와 타입 확인
print('숙박소비내역.csv의 구조와 타입 : ')
print(lodge_consume_new.info())

숙박소비내역.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118 entries, 0 to 1117
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   TRAVEL_ID         1118 non-null   object
 1   가족/친지집            1118 non-null   int64 
 2   게스트하우스            1118 non-null   int64 
 3   기타                1118 non-null   int64 
 4   모텔/여관             1118 non-null   int64 
 5   민박                1118 non-null   int64 
 6   야영(캠핑,글램핑등)       1118 non-null   int64 
 7   자연휴양림내 시설         1118 non-null   int64 
 8   전통 숙박시설           1118 non-null   int64 
 9   콘도미니엄/리조트         1118 non-null   int64 
 10  펜션                1118 non-null   int64 
 11  호텔                1118 non-null   int64 
 12  MAJOR_LODGE_TYPE  1118 non-null   object
 13  TOTAL_LODGE_WON   1118 non-null   int64 
dtypes: int64(12), object(2)
memory usage: 122.4+ KB
None


In [1665]:
# 데이터의 결측치 확인
print('숙박소비내역.csv의 결측치 : ')
print(lodge_consume_new.isnull().sum())

숙박소비내역.csv의 결측치 : 
TRAVEL_ID           0
가족/친지집              0
게스트하우스              0
기타                  0
모텔/여관               0
민박                  0
야영(캠핑,글램핑등)         0
자연휴양림내 시설           0
전통 숙박시설             0
콘도미니엄/리조트           0
펜션                  0
호텔                  0
MAJOR_LODGE_TYPE    0
TOTAL_LODGE_WON     0
dtype: int64


In [1666]:
# 데이터 살펴보기
lodge_consume_new.head()

,TRAVEL_ID,가족/친지집,게스트하우스,기타,모텔/여관,민박,"야영(캠핑,글램핑등)",자연휴양림내 시설,전통 숙박시설,콘도미니엄/리조트,펜션,호텔,MAJOR_LODGE_TYPE,TOTAL_LODGE_WON
0,e_e000005,0,0,0,87000,0,0,0,0,121440,0,90000,"[모텔/여관, 콘도미니엄/리조트, 호텔]",298440
1,e_e000191,0,0,0,0,0,0,0,0,0,240000,0,[펜션],240000
2,e_e000316,0,0,0,131095,0,0,0,0,0,0,0,[모텔/여관],131095
3,e_e004870,0,0,0,0,0,0,260000,0,0,0,0,[자연휴양림내 시설],260000
4,f_f000001,0,0,0,0,0,100000,0,0,0,0,0,"[야영(캠핑,글램핑등)]",100000


<hr>
<h2>[활동내역.csv]</h2>

<h3>1. 변수 추출 및 검토</h3>

In [1667]:
# 필요한 변수 리스트
columns_to_keep_activity = [
    'TRAVEL_ID', 'ACTIVITY_TYPE_CD'
    ]

# activity에 필요한 변수만 남기고 저장
activity = activity[columns_to_keep_activity]

# 결과 확인
print(activity.head())

   TRAVEL_ID  ACTIVITY_TYPE_CD
0  e_e000005                 1
1  e_e000005                 1
2  e_e000005                 1
3  e_e000005                 1
4  e_e000005                 1


In [1668]:
# 데이터의 행, 열 개수 출력
print('활동내역.csv의 행, 열 개수 : ')
print(activity.shape)

활동내역.csv의 행, 열 개수 : 
(32218, 2)


In [1669]:
# 데이터의 구조와 타입 확인
print('활동내역.csv의 구조와 타입 : ')
print(activity.info())

활동내역.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32218 entries, 0 to 32217
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   TRAVEL_ID         32218 non-null  object
 1   ACTIVITY_TYPE_CD  32218 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 503.5+ KB
None


In [1670]:
# 데이터의 결측치 확인
print('활동내역.csv의 결측치 : ')
print(activity.isnull().sum())

활동내역.csv의 결측치 : 
TRAVEL_ID           0
ACTIVITY_TYPE_CD    0
dtype: int64


In [1671]:
# 데이터 살펴보기
activity.head()

,TRAVEL_ID,ACTIVITY_TYPE_CD
0,e_e000005,1
1,e_e000005,1
2,e_e000005,1
3,e_e000005,1
4,e_e000005,1


<h3>2. 전처리</h3>

<h5>1) ACTIVITY_TYPE_CD 변수 코드 'ACT' 매핑</h5>

In [1672]:
activity = convert_codes_to_korean(activity, 'ACTIVITY_TYPE_CD', ACT_mapping)

# 결과 확인
activity.head()

,TRAVEL_ID,ACTIVITY_TYPE_CD
0,e_e000005,취식
1,e_e000005,취식
2,e_e000005,취식
3,e_e000005,취식
4,e_e000005,취식


<h5>2) 추가적인 분석에 사용하기 위해 TRAVEL_ID별 주요활동유형 구하기

In [1673]:
# TRAVEL_ID별로 활동유형 빈도 계산
activity_count = activity.groupby(['TRAVEL_ID', 'ACTIVITY_TYPE_CD']).size().reset_index(name='COUNT')

# TRAVEL_ID별로 가장 많은 활동유형 추출
activity_new = activity_count.groupby('TRAVEL_ID').apply(
    lambda x: x[x['COUNT'] == x['COUNT'].max()]['ACTIVITY_TYPE_CD'].tolist()  # 빈도수가 동일한 활동유형을 리스트로 결합
).reset_index(name='MAJOR_ACTIVITY_TYPE')

# 결과 확인
print(activity_new.head())

   TRAVEL_ID MAJOR_ACTIVITY_TYPE
0  e_e000005                [취식]
1  e_e000043   [단순 구경 / 산책 / 걷기]
2  e_e000191                [취식]
3  e_e000316                [취식]
4  e_e000668           [쇼핑 / 구매]


<h3>3. 전처리 후 확인</h3>

In [1674]:
# TRAVEL_ID 중복 여부 확인
duplicates = activity_new[activity_new.duplicated(subset='TRAVEL_ID', keep=False)]

# 중복된 항목이 있는 경우
if not duplicates.empty:
    print("중복된 TRAVEL_ID가 있습니다.")
else:
    print("중복된 TRAVEL_ID가 없습니다.")

중복된 TRAVEL_ID가 없습니다.


In [1675]:
# 데이터의 구조와 타입 확인
print('활동내역.csv의 구조와 타입 : ')
print(activity_new.info())

활동내역.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2560 entries, 0 to 2559
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   TRAVEL_ID            2560 non-null   object
 1   MAJOR_ACTIVITY_TYPE  2560 non-null   object
dtypes: object(2)
memory usage: 40.1+ KB
None


In [1676]:
# 데이터의 결측치 확인
print('활동내역.csv의 결측치 : ')
print(activity_new.isnull().sum())

활동내역.csv의 결측치 : 
TRAVEL_ID              0
MAJOR_ACTIVITY_TYPE    0
dtype: int64


In [1677]:
# 데이터 살펴보기
activity_new.head()

,TRAVEL_ID,MAJOR_ACTIVITY_TYPE
0,e_e000005,[취식]
1,e_e000043,[단순 구경 / 산책 / 걷기]
2,e_e000191,[취식]
3,e_e000316,[취식]
4,e_e000668,[쇼핑 / 구매]


<hr>
<h2>[활동소비내역.csv]</h2>

<h3>1. 변수 추출 및 검토</h3>

In [1678]:
# 필요한 변수 리스트
columns_to_keep_activity_consume = [
    'TRAVEL_ID', 'PAYMENT_AMT_WON'
    ]

# activity_counsume에 필요한 변수만 남기고 저장
activity_consume = activity_consume[columns_to_keep_activity_consume]

# 결과 확인
print(activity_consume.head())

   TRAVEL_ID  PAYMENT_AMT_WON
0  g_g000540             0.00
1  g_g001133        24,000.00
2  g_g001133        12,000.00
3  g_g000638        14,000.00
4  g_g002127        22,000.00


In [1679]:
# 데이터의 행, 열 개수 출력
print('활동소비내역.csv의 행, 열 개수 : ')
print(activity_consume.shape)

활동소비내역.csv의 행, 열 개수 : 
(15841, 2)


In [1680]:
# 데이터의 구조와 타입 확인
print('활동소비내역.csv의 구조와 타입 : ')
print(activity_consume.info())

활동소비내역.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15841 entries, 0 to 15840
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TRAVEL_ID        15841 non-null  object 
 1   PAYMENT_AMT_WON  15839 non-null  float64
dtypes: float64(1), object(1)
memory usage: 247.6+ KB
None


In [1681]:
# 데이터의 결측치 확인
print('활동소비내역.csv의 결측치 : ')
print(activity_consume.isnull().sum())

활동소비내역.csv의 결측치 : 
TRAVEL_ID          0
PAYMENT_AMT_WON    2
dtype: int64


In [1682]:
# 결측치가 있는 행을 확인
missing_data_rows = activity_consume[activity_consume.isnull().any(axis=1)]

# 결측치가 있는 행 출력
print(missing_data_rows)

      TRAVEL_ID  PAYMENT_AMT_WON
4875  g_g002220              NaN
7996  g_g004485              NaN


In [1683]:
# 결측치 제거 
activity_consume.dropna(inplace=True)

# 결측치 제거 확인
print('활동소비내역.csv의 결측치 : ')
print(activity_consume.isnull().sum())

활동소비내역.csv의 결측치 : 
TRAVEL_ID          0
PAYMENT_AMT_WON    0
dtype: int64


> 결측치 분석 결과 : 원본 데이터 확인한 결과, 결측치는 두 가지 이유로 발생
<br> 1. 행이 모두 비어 있는 경우
<br> 2. 통합권이라서 다른 데이터에 값이 포함되어 있는 경우
<br> 따라서, 해당 결측치는 드롭(drop) 처리 함

In [1684]:
# 데이터 살펴보기
activity_consume.head()

,TRAVEL_ID,PAYMENT_AMT_WON
0,g_g000540,0.00
1,g_g001133,"24,000.00"
2,g_g001133,"12,000.00"
3,g_g000638,"14,000.00"
4,g_g002127,"22,000.00"


<h3>2. 전처리</h3>

<h5>1) 활동 소비내역인 걸 나타내기 위해서 변수명 변경

In [1685]:
activity_consume = activity_consume.rename(columns = {'PAYMENT_AMT_WON' : 'ACTIVITY_WON'})

# 결과 확인
activity_consume.head()

,TRAVEL_ID,ACTIVITY_WON
0,g_g000540,0.00
1,g_g001133,"24,000.00"
2,g_g001133,"12,000.00"
3,g_g000638,"14,000.00"
4,g_g002127,"22,000.00"


<h5>2) 추가적인 분석에 사용하기 위해 TRAVEL_ID별 활동총소비액 파생변수 추가

In [ ]:
# 결과를 저장하기 위한 activity_consume_new 데이터프레임 준비
activity_consume_new = activity_consume.drop(columns=['ACTIVITY_WON']).drop_duplicates(subset=['TRAVEL_ID'])

# TRAVEL_ID별로 이동수단의 총 소비액 계산
activity_consume_total = activity_consume.groupby('TRAVEL_ID')['ACTIVITY_WON'].sum().reset_index(name='TOTAL_ACTIVITY_WON')

# activity_consume_new 라는 새로운 데이터프레임에 저장
activity_consume_new = activity_consume_new.merge(activity_consume_total, on='TRAVEL_ID', how='left')

# 결과 확인
print(activity_consume_new.head())

   TRAVEL_ID  TOTAL_ACTIVITY_WON
0  g_g000540           86,200.00
1  g_g001133          284,600.00
2  g_g000638          341,480.00
3  g_g002127          210,350.00
4  g_g006975          359,796.00


<h3>3. 전처리 후 확인</h3>

In [1687]:
# TRAVEL_ID 중복 여부 확인
duplicates = activity_consume_new[activity_consume_new.duplicated(subset='TRAVEL_ID', keep=False)]

# 중복된 항목이 있는 경우
if not duplicates.empty:
    print("중복된 TRAVEL_ID가 있습니다.")
else:
    print("중복된 TRAVEL_ID가 없습니다.")

중복된 TRAVEL_ID가 없습니다.


In [1688]:
# 데이터의 구조와 타입 확인
print('활동소비내역.csv의 구조와 타입 : ')
print(activity_consume_new.info())

활동소비내역.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2547 entries, 0 to 2546
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TRAVEL_ID           2547 non-null   object 
 1   TOTAL_ACTIVITY_WON  2547 non-null   float64
dtypes: float64(1), object(1)
memory usage: 39.9+ KB
None


In [1689]:
# 데이터의 결측치 확인
print('활동소비내역.csv의 결측치 : ')
print(activity_consume_new.isnull().sum())

활동소비내역.csv의 결측치 : 
TRAVEL_ID             0
TOTAL_ACTIVITY_WON    0
dtype: int64


In [1690]:
# 데이터 살펴보기
activity_consume_new.head()

,TRAVEL_ID,TOTAL_ACTIVITY_WON
0,g_g000540,"86,200.00"
1,g_g001133,"284,600.00"
2,g_g000638,"341,480.00"
3,g_g002127,"210,350.00"
4,g_g006975,"359,796.00"


<hr>
<h2>[사전소비내역.csv]</h2>

<h3>1. 변수 추출 및 검토</h3>

In [1691]:
# 필요한 변수 리스트
columns_to_keep_adv_consume = [
    'TRAVEL_ID', 'ADV_NM', 'PAYMENT_AMT_WON'
    ]

# adv_consume에 필요한 변수만 남기고 저장
adv_consume = adv_consume[columns_to_keep_adv_consume]

# 결과 확인
print(adv_consume.head())

   TRAVEL_ID   ADV_NM  PAYMENT_AMT_WON
0  g_g001280  플로깅용 집게             4000
1  g_g004822     빵 커피             4000
2  g_g001276    햇반 라면             6000
3  g_g000249       간식             7000
4  g_g000365       공차            10000


In [1692]:
# 데이터의 행, 열 개수 출력
print('사전소비내역.csv의 행, 열 개수 : ')
print(adv_consume.shape)

사전소비내역.csv의 행, 열 개수 : 
(534, 3)


In [1693]:
# 데이터의 구조와 타입 확인
print('사전소비내역.csv의 구조와 타입 : ')
print(adv_consume.info())

사전소비내역.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   TRAVEL_ID        534 non-null    object
 1   ADV_NM           534 non-null    object
 2   PAYMENT_AMT_WON  534 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 12.6+ KB
None


In [1694]:
# 데이터의 결측치 확인
print('사전소비내역.csv의 결측치 : ')
print(adv_consume.isnull().sum())

사전소비내역.csv의 결측치 : 
TRAVEL_ID          0
ADV_NM             0
PAYMENT_AMT_WON    0
dtype: int64


In [1695]:
# 데이터 살펴보기
adv_consume.head()

,TRAVEL_ID,ADV_NM,PAYMENT_AMT_WON
0,g_g001280,플로깅용 집게,4000
1,g_g004822,빵 커피,4000
2,g_g001276,햇반 라면,6000
3,g_g000249,간식,7000
4,g_g000365,공차,10000


<h3>2. 전처리</h3>

<h5>1) 이동수단 소비내역인 걸 나타내기 위해서 변수명 변경

In [1696]:
mvmn_consume = mvmn_consume.rename(columns = {'PAYMENT_AMT_WON' : 'MVMN_WON'})

# 결과 확인
mvmn_consume.head()

,TRAVEL_ID,MVMN_SE_NM,MVMN_WON
0,g_g007947,렌터카(승용/승합/버스 등등),1500
1,g_g000552,기타,1500
2,g_g000419,시외/고속버스,2000
3,g_g010747,택시,2000
4,g_g007226,시내/마을버스,2500


<h5>2) 추가적인 분석에 사용하기 위해 TRAVEL_ID별 주요이동수단 구하기

In [1697]:
# TRAVEL_ID별로 이동수단 빈도 계산
mvmn_consume_count = mvmn_consume.groupby(['TRAVEL_ID', 'MVMN_SE_NM']).size().reset_index(name='COUNT')

# TRAVEL_ID별로 가장 많이 사용된 이동수단 추출
mvmn_consume_major_transport = mvmn_consume_count.groupby('TRAVEL_ID').apply(
    lambda x: x[x['COUNT'] == x['COUNT'].max()]['MVMN_SE_NM'].tolist()  # 빈도수가 동일한 이동수단을 리스트로 결합
).reset_index(name='MAJOR_TRANSPORT')

# 결과 확인
print(mvmn_consume_major_transport.head())

   TRAVEL_ID     MAJOR_TRANSPORT
0  e_e000005  [자가용(승용/승합/트럭 등등)]
1  e_e000043  [자가용(승용/승합/트럭 등등)]
2  e_e000191  [자가용(승용/승합/트럭 등등)]
3  e_e000316                [택시]
4  e_e000668  [자가용(승용/승합/트럭 등등)]


<h5>3) TRAVEL_ID가 중복되는 경우 이동수단 소비내역 전처리 - pivot_table

In [1698]:
# TRAVEL_ID 중복 여부 확인
duplicates = mvmn_consume[mvmn_consume.duplicated(subset='TRAVEL_ID', keep=False)]

# 중복된 항목이 있는 경우
if not duplicates.empty:
    print("중복된 TRAVEL_ID가 있습니다.")
else:
    print("중복된 TRAVEL_ID가 없습니다.")

중복된 TRAVEL_ID가 있습니다.


pivot_table 형식으로 변환하여, 이후 파생변수 추가 등 추가적인 분석을 쉽게 할 수 있도록 함

In [1699]:
# MVMN_SE_NM을 문자열 형식으로 변환
mvmn_consume['MVMN_SE_NM'] = mvmn_consume['MVMN_SE_NM'].astype(str)

# pivot_table 생성
mvmn_consume_pivot = mvmn_consume.pivot_table(
    index='TRAVEL_ID', 
    columns='MVMN_SE_NM', 
    values='MVMN_WON', 
    aggfunc='sum', 
    fill_value=0  # 결측값은 0으로
)

# 결과 확인
print(mvmn_consume_pivot.head())

MVMN_SE_NM  KTX/SRT(고속열차)  관광버스  기타  렌터카(승용/승합/버스 등등)  배/선박  버스 + 지하철  새마을/무궁화열차  시내/마을버스  시외/고속버스  자가용(승용/승합/트럭 등등)  자전거  지하철  캠핑카(자차 및 렌탈)     택시  항공기
TRAVEL_ID                                                                                                                                               
e_e000005               0     0   0                 0     0         0          0        0        0            112300    0    0             0      0    0
e_e000043               0     0   0                 0     0         0          0        0        0             42925    0    0             0      0    0
e_e000191               0     0   0                 0     0         0          0        0        0             98000    0    0             0      0    0
e_e000316          140000     0   0                 0     0         0          0    40000        0              9000    0    0             0  15300    0
e_e000668               0     0   0                 0     0         0          0  

2)에서 구했던 주요이동수단과 병합하여 mvmn_consume_new 데이터프레임 생성

In [1700]:
# 주요 이동수단과 피벗 테이블 병합
mvmn_consume_new = pd.merge(
    mvmn_consume_pivot.reset_index(),  # 피벗 테이블을 병합하기 위해 인덱스를 초기화
    mvmn_consume_major_transport,     # 주요 이동수단 데이터프레임
    on='TRAVEL_ID',                   # 공통 키: TRAVEL_ID
    how='left'                        # 병합 방식: 왼쪽 (피벗 테이블 기준)
)

# 결과 확인
print(mvmn_consume_new.head())

   TRAVEL_ID  KTX/SRT(고속열차)  관광버스  기타  렌터카(승용/승합/버스 등등)  배/선박  버스 + 지하철  새마을/무궁화열차  시내/마을버스  시외/고속버스  자가용(승용/승합/트럭 등등)  자전거  지하철  캠핑카(자차 및 렌탈)     택시  항공기     MAJOR_TRANSPORT
0  e_e000005              0     0   0                 0     0         0          0        0        0            112300    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]
1  e_e000043              0     0   0                 0     0         0          0        0        0             42925    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]
2  e_e000191              0     0   0                 0     0         0          0        0        0             98000    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]
3  e_e000316         140000     0   0                 0     0         0          0    40000        0              9000    0    0             0  15300    0                [택시]
4  e_e000668              0     0   0                 0     0         0          0        0        0             86400    0  

<h5>4) 추가적인 분석에 사용하기 위해 TRAVEL_ID별 이동수단총소비액 파생변수 추가

In [1701]:
# TRAVEL_ID별로 이동수단의 총 소비액 계산
mvmn_consume_total = mvmn_consume.groupby('TRAVEL_ID')['MVMN_WON'].sum().reset_index(name='TOTAL_MVMN_WON')

# mvmn_consume_new와 병합
mvmn_consume_new = mvmn_consume_new.merge(mvmn_consume_total, on='TRAVEL_ID', how='left')

# 결과 확인
print(mvmn_consume_new.head())

   TRAVEL_ID  KTX/SRT(고속열차)  관광버스  기타  렌터카(승용/승합/버스 등등)  배/선박  버스 + 지하철  새마을/무궁화열차  시내/마을버스  시외/고속버스  자가용(승용/승합/트럭 등등)  자전거  지하철  캠핑카(자차 및 렌탈)     택시  항공기     MAJOR_TRANSPORT  TOTAL_MVMN_WON
0  e_e000005              0     0   0                 0     0         0          0        0        0            112300    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]          112300
1  e_e000043              0     0   0                 0     0         0          0        0        0             42925    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]           42925
2  e_e000191              0     0   0                 0     0         0          0        0        0             98000    0    0             0      0    0  [자가용(승용/승합/트럭 등등)]           98000
3  e_e000316         140000     0   0                 0     0         0          0    40000        0              9000    0    0             0  15300    0                [택시]          204300
4  e_e000668              0     0   0        

<h5>2) JOB_NM 변수 코드 'JOB' 매핑</h5>

<h3>3. 전처리 후 확인</h3>

<hr>
<h2>[방문지정보.csv]</h2>

<h3>1. 변수 추출 및 검토</h3>

In [1702]:
# 필요한 변수 리스트
columns_to_keep_visit_area = [
    'TRAVEL_ID', 'RESIDENCE_TIME_MIN', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 
    'VISIT_CHC_REASON_CD', 'LODGING_TYPE_CD', 'DGSTFN', 'REVISIT_INTENTION'
    ]

# visit_area에 필요한 변수만 남기고 저장
visit_area = visit_area[columns_to_keep_visit_area]

# 결과 확인
print(visit_area.head())

   TRAVEL_ID  RESIDENCE_TIME_MIN  VISIT_AREA_TYPE_CD REVISIT_YN  VISIT_CHC_REASON_CD  LODGING_TYPE_CD  DGSTFN  REVISIT_INTENTION
0  e_e000005                 NaN                  21        NaN                  NaN              NaN     NaN                NaN
1  e_e000005               30.00                  11          N                 5.00              NaN    5.00               5.00
2  e_e000005               30.00                   9          Y                 6.00              NaN    5.00               5.00
3  e_e000005               30.00                  24          N                 8.00             2.00    5.00               5.00
4  e_e000005               60.00                   1          N                 1.00              NaN    5.00               5.00


In [1703]:
# 데이터의 행, 열 개수 출력
print('방문지정보.csv의 행, 열 개수 : ')
print(visit_area)

방문지정보.csv의 행, 열 개수 : 
       TRAVEL_ID  RESIDENCE_TIME_MIN  VISIT_AREA_TYPE_CD REVISIT_YN  VISIT_CHC_REASON_CD  LODGING_TYPE_CD  DGSTFN  REVISIT_INTENTION
0      e_e000005                 NaN                  21        NaN                  NaN              NaN     NaN                NaN
1      e_e000005               30.00                  11          N                 5.00              NaN    5.00               5.00
2      e_e000005               30.00                   9          Y                 6.00              NaN    5.00               5.00
3      e_e000005               30.00                  24          N                 8.00             2.00    5.00               5.00
4      e_e000005               60.00                   1          N                 1.00              NaN    5.00               5.00
...          ...                 ...                 ...        ...                  ...              ...     ...                ...
28384  h_h003280               90.00           

In [1704]:
# 데이터의 구조와 타입 확인
print('방문지정보.csv의 구조와 타입 : ')
print(visit_area.info())

방문지정보.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28389 entries, 0 to 28388
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TRAVEL_ID            28389 non-null  object 
 1   RESIDENCE_TIME_MIN   25826 non-null  float64
 2   VISIT_AREA_TYPE_CD   28389 non-null  int64  
 3   REVISIT_YN           22817 non-null  object 
 4   VISIT_CHC_REASON_CD  22832 non-null  float64
 5   LODGING_TYPE_CD      2243 non-null   float64
 6   DGSTFN               22813 non-null  float64
 7   REVISIT_INTENTION    22813 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 1.7+ MB
None


In [1705]:
# 데이터의 결측치 확인
print('방문지정보.csv의 결측치 : ')
print(visit_area.isnull().sum())

방문지정보.csv의 결측치 : 
TRAVEL_ID                  0
RESIDENCE_TIME_MIN      2563
VISIT_AREA_TYPE_CD         0
REVISIT_YN              5572
VISIT_CHC_REASON_CD     5557
LODGING_TYPE_CD        26146
DGSTFN                  5576
REVISIT_INTENTION       5576
dtype: int64


In [1706]:
# 데이터 살펴보기
visit_area.head()

,TRAVEL_ID,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION
0,e_e000005,NaN,21,NaN,NaN,NaN,NaN,NaN
1,e_e000005,30.00,11,N,5.00,NaN,5.00,5.00
2,e_e000005,30.00,9,Y,6.00,NaN,5.00,5.00
3,e_e000005,30.00,24,N,8.00,2.00,5.00,5.00
4,e_e000005,60.00,1,N,1.00,NaN,5.00,5.00


<h5>2) JOB_NM 변수 코드 'JOB' 매핑</h5>

<h3>3. 전처리 후 확인</h3>

<hr>
<h3>[데이터 병합]</h3>